<a href="https://colab.research.google.com/github/JohnSmith69969/ADS2002-Monash-Solar-Group/blob/Max/CSV_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from datetime import datetime
from distutils.util import strtobool

import pandas as pd



# Converts the contents in a .tsf file into a dataframe and returns it along with other meta-data of the dataset: frequency, horizon, whether the dataset contains missing values and whether the series have equal lengths
#
# Parameters
# full_file_path_and_name - complete .tsf file path
# replace_missing_vals_with - a term to indicate the missing values in series in the returning dataframe
# value_column_name - Any name that is preferred to have as the name of the column containing series values in the returning dataframe
def convert_tsf_to_dataframe(
    full_file_path_and_name,
    replace_missing_vals_with="NaN",
    value_column_name="series_value",
):
    col_names = []
    col_types = []
    all_data = {}
    line_count = 0
    frequency = None
    forecast_horizon = None
    contain_missing_values = None
    contain_equal_length = None
    found_data_tag = False
    found_data_section = False
    started_reading_data_section = False

    with open(full_file_path_and_name, "r", encoding="cp1252") as file:
        for line in file:
            # Strip white space from start/end of line
            line = line.strip()

            if line:
                if line.startswith("@"):  # Read meta-data
                    if not line.startswith("@data"):
                        line_content = line.split(" ")
                        if line.startswith("@attribute"):
                            if (
                                len(line_content) != 3
                            ):  # Attributes have both name and type
                                raise Exception("Invalid meta-data specification.")

                            col_names.append(line_content[1])
                            col_types.append(line_content[2])
                        else:
                            if (
                                len(line_content) != 2
                            ):  # Other meta-data have only values
                                raise Exception("Invalid meta-data specification.")

                            if line.startswith("@frequency"):
                                frequency = line_content[1]
                            elif line.startswith("@horizon"):
                                forecast_horizon = int(line_content[1])
                            elif line.startswith("@missing"):
                                contain_missing_values = bool(
                                    strtobool(line_content[1])
                                )
                            elif line.startswith("@equallength"):
                                contain_equal_length = bool(strtobool(line_content[1]))

                    else:
                        if len(col_names) == 0:
                            raise Exception(
                                "Missing attribute section. Attribute section must come before data."
                            )

                        found_data_tag = True
                elif not line.startswith("#"):
                    if len(col_names) == 0:
                        raise Exception(
                            "Missing attribute section. Attribute section must come before data."
                        )
                    elif not found_data_tag:
                        raise Exception("Missing @data tag.")
                    else:
                        if not started_reading_data_section:
                            started_reading_data_section = True
                            found_data_section = True
                            all_series = []

                            for col in col_names:
                                all_data[col] = []

                        full_info = line.split(":")

                        if len(full_info) != (len(col_names) + 1):
                            raise Exception("Missing attributes/values in series.")

                        series = full_info[len(full_info) - 1]
                        series = series.split(",")

                        if len(series) == 0:
                            raise Exception(
                                "A given series should contains a set of comma separated numeric values. At least one numeric value should be there in a series. Missing values should be indicated with ? symbol"
                            )

                        numeric_series = []

                        for val in series:
                            if val == "?":
                                numeric_series.append(replace_missing_vals_with)
                            else:
                                numeric_series.append(float(val))

                        if numeric_series.count(replace_missing_vals_with) == len(
                            numeric_series
                        ):
                            raise Exception(
                                "All series values are missing. A given series should contains a set of comma separated numeric values. At least one numeric value should be there in a series."
                            )

                        all_series.append(pd.Series(numeric_series).array)

                        for i in range(len(col_names)):
                            att_val = None
                            if col_types[i] == "numeric":
                                att_val = int(full_info[i])
                            elif col_types[i] == "string":
                                att_val = str(full_info[i])
                            elif col_types[i] == "date":
                                att_val = datetime.strptime(
                                    full_info[i], "%Y-%m-%d %H-%M-%S"
                                )
                            else:
                                raise Exception(
                                    "Invalid attribute type."
                                )  # Currently, the code supports only numeric, string and date types. Extend this as required.

                            if att_val is None:
                                raise Exception("Invalid attribute value.")
                            else:
                                all_data[col_names[i]].append(att_val)

                line_count = line_count + 1

        if line_count == 0:
            raise Exception("Empty file.")
        if len(col_names) == 0:
            raise Exception("Missing attribute section.")
        if not found_data_section:
            raise Exception("Missing series information under data section.")

        all_data[value_column_name] = all_series
        loaded_data = pd.DataFrame(all_data)

        return (
            loaded_data,
            frequency,
            forecast_horizon,
            contain_missing_values,
            contain_equal_length,
        )


#loaded_data, frequency, forecast_horizon, contain_missing_values, contain_equal_length = convert_tsf_to_dataframe("TSForecasting/tsf_data/sample.tsf")

#print(loaded_data)
#print(frequency)
#print(forecast_horizon)
#print(contain_missing_values)
#print(contain_equal_length)

In [2]:
loaded_data, frequency, forecast_horizon, contain_missing_values, contain_equal_length = convert_tsf_to_dataframe('/content/drive/MyDrive/Solar Farm Data/phase_1_data.tsf')
loaded_data.head()

,series_name,start_timestamp,series_value
0,Building0,2016-07-03 21:30:00,"[283.8, 283.8, 283.8, 606.0, 606.0, 606.0, 606..."
1,Building1,2019-01-09 23:15:00,"[8.1, 15.7, 22.8, 32.7, 8.1, 16.5, 24.7, 34.5,..."
2,Building3,2016-03-01 04:15:00,"[1321.0, 1321.0, 1321.0, 1321.0, 1293.0, 1293...."
3,Building4,2019-07-03 04:45:00,"[2.0, NaN, 1.0, 2.0, NaN, 2.0, NaN, NaN, 2.0, ..."
4,Building5,2019-07-25 23:00:00,"[30.0, 31.0, 24.0, 34.0, 30.0, 31.0, 26.0, 33...."


In [3]:
import pandas as pd
import numpy as np
phase1_df = pd.DataFrame()
for i in range(len(loaded_data)):
    phase1_df[loaded_data['series_name'][i]] = pd.Series(data=list(loaded_data['series_value'][i]),
                                                    index=pd.date_range(loaded_data['start_timestamp'][i],
                                                    periods=len(loaded_data['series_value'][i]),
                                                    freq="15min")).astype(float)
phase1_df.index.names = ['Date']
phase1_df.head()

,Building0,Building1,Building3,Building4,Building5,Building6,Solar0,Solar1,Solar2,Solar3,Solar4,Solar5
Date,,,,,,,,,,,,
2016-07-03 21:30:00,283.8,NaN,581.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-03 21:45:00,283.8,NaN,581.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-03 22:00:00,283.8,NaN,581.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-03 22:15:00,606.0,NaN,624.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-03 22:30:00,606.0,NaN,624.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
phase1_df.describe()

,Building0,Building1,Building3,Building4,Building5,Building6,Solar0,Solar1,Solar2,Solar3,Solar4,Solar5
count,101406.000000,60395.000000,148251.000000,26096.000000,11580.000000,39402.000000,15208.000000,61388.000000,46408.000000,46408.000000,46408.000000,59948.000000
mean,230.108545,11.374756,520.273550,1.327866,24.666062,30.705822,4.386435,1.937350,1.877698,1.358164,1.117320,3.890990
std,133.536120,7.910689,273.979951,0.530892,11.103474,5.593896,8.858768,3.434536,3.123851,2.213344,1.893412,8.745043
min,0.100000,1.500000,85.000000,1.000000,1.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,168.300000,5.700000,310.000000,1.000000,18.000000,26.600000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,214.850000,9.400000,394.000000,1.000000,21.000000,28.600000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,277.100000,14.500000,724.000000,2.000000,36.000000,35.000000,4.130000,2.410000,3.370000,1.720000,1.550000,0.970000
max,6781.500000,83.500000,8556.000000,5.000000,62.000000,102.200000,50.410000,13.040000,13.960000,11.040000,8.100000,40.430000


In [5]:
phase1_df.to_csv('/content/drive/MyDrive/Solar Farm Data/CSV data/phase1.csv')

In [6]:
loaded_data, frequency, forecast_horizon, contain_missing_values, contain_equal_length = convert_tsf_to_dataframe('/content/drive/MyDrive/Solar Farm Data/phase_2_data.tsf')
loaded_data.head()

,series_name,start_timestamp,series_value
0,Building0,2016-07-03 21:30:00,"[283.8, 283.8, 283.8, 606.0, 606.0, 606.0, 606..."
1,Building1,2019-01-09 23:15:00,"[8.1, 15.7, 22.8, 32.7, 8.1, 16.5, 24.7, 34.5,..."
2,Building3,2016-03-01 04:15:00,"[1321.0, 1321.0, 1321.0, 1321.0, 1293.0, 1293...."
3,Building4,2019-07-03 04:45:00,"[2.0, NaN, 1.0, 2.0, NaN, 2.0, NaN, NaN, 2.0, ..."
4,Building5,2019-07-25 23:00:00,"[30.0, 31.0, 24.0, 34.0, 30.0, 31.0, 26.0, 33...."


In [7]:
phase2_df = pd.DataFrame()
for i in range(len(loaded_data)):
    phase2_df[loaded_data['series_name'][i]] = pd.Series(data=list(loaded_data['series_value'][i]),
                                                    index=pd.date_range(loaded_data['start_timestamp'][i],
                                                    periods=len(loaded_data['series_value'][i]),
                                                    freq="15min")).astype(float)
phase2_df.index.names = ['Date']
phase2_df.head()

,Building0,Building1,Building3,Building4,Building5,Building6,Solar0,Solar1,Solar2,Solar3,Solar4,Solar5
Date,,,,,,,,,,,,
2016-07-03 21:30:00,283.8,NaN,581.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-03 21:45:00,283.8,NaN,581.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-03 22:00:00,283.8,NaN,581.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-03 22:15:00,606.0,NaN,624.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-03 22:30:00,606.0,NaN,624.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
phase2_df.to_csv('/content/drive/MyDrive/Solar Farm Data/CSV data/phase2.csv')

In [9]:
loaded_data, frequency, forecast_horizon, contain_missing_values, contain_equal_length = convert_tsf_to_dataframe('/content/drive/MyDrive/Solar Farm Data/nov_data.tsf')
loaded_data.head()

,series_name,start_timestamp,series_value
0,Building0,2016-07-03 21:30:00,"[283.8, 283.8, 283.8, 606.0, 606.0, 606.0, 606..."
1,Building1,2019-01-09 23:15:00,"[8.1, 15.7, 22.8, 32.7, 8.1, 16.5, 24.7, 34.5,..."
2,Building3,2016-03-01 04:15:00,"[1321.0, 1321.0, 1321.0, 1321.0, 1293.0, 1293...."
3,Building4,2019-07-03 04:45:00,"[2.0, NaN, 1.0, 2.0, NaN, 2.0, NaN, NaN, 2.0, ..."
4,Building5,2019-07-25 23:00:00,"[30.0, 31.0, 24.0, 34.0, 30.0, 31.0, 26.0, 33...."


In [10]:
finaltest_df = pd.DataFrame()
for i in range(len(loaded_data)):
    finaltest_df[loaded_data['series_name'][i]] = pd.Series(data=list(loaded_data['series_value'][i]),
                                                    index=pd.date_range(loaded_data['start_timestamp'][i],
                                                    periods=len(loaded_data['series_value'][i]),
                                                    freq="15min")).astype(float)
finaltest_df.index.names = ['Date']
finaltest_df.head()

,Building0,Building1,Building3,Building4,Building5,Building6,Solar0,Solar1,Solar2,Solar3,Solar4,Solar5
Date,,,,,,,,,,,,
2016-07-03 21:30:00,283.8,NaN,581.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-03 21:45:00,283.8,NaN,581.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-03 22:00:00,283.8,NaN,581.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-03 22:15:00,606.0,NaN,624.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-07-03 22:30:00,606.0,NaN,624.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
finaltest_df.to_csv('/content/drive/MyDrive/Solar Farm Data/CSV data/finaltest.csv')

In [13]:
phase1_df.describe()

,Building0,Building1,Building3,Building4,Building5,Building6,Solar0,Solar1,Solar2,Solar3,Solar4,Solar5
count,101406.000000,60395.000000,148251.000000,26096.000000,11580.000000,39402.000000,15208.000000,61388.000000,46408.000000,46408.000000,46408.000000,59948.000000
mean,230.108545,11.374756,520.273550,1.327866,24.666062,30.705822,4.386435,1.937350,1.877698,1.358164,1.117320,3.890990
std,133.536120,7.910689,273.979951,0.530892,11.103474,5.593896,8.858768,3.434536,3.123851,2.213344,1.893412,8.745043
min,0.100000,1.500000,85.000000,1.000000,1.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,168.300000,5.700000,310.000000,1.000000,18.000000,26.600000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,214.850000,9.400000,394.000000,1.000000,21.000000,28.600000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,277.100000,14.500000,724.000000,2.000000,36.000000,35.000000,4.130000,2.410000,3.370000,1.720000,1.550000,0.970000
max,6781.500000,83.500000,8556.000000,5.000000,62.000000,102.200000,50.410000,13.040000,13.960000,11.040000,8.100000,40.430000


In [14]:
phase2_df.describe()

,Building0,Building1,Building3,Building4,Building5,Building6,Solar0,Solar1,Solar2,Solar3,Solar4,Solar5
count,104320.000000,63371.000000,151227.000000,27787.000000,11713.000000,42374.000000,18184.000000,64364.000000,49384.000000,49384.000000,49384.000000,62924.000000
mean,226.813641,11.278787,516.793390,1.323461,24.643473,30.660339,5.239122,1.964903,1.913878,1.412631,1.138822,4.072200
std,133.732085,7.822719,272.728823,0.527606,11.102675,5.620018,10.012907,3.446606,3.156583,2.297215,1.911003,8.911772
min,0.100000,1.500000,85.000000,1.000000,1.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,166.000000,5.600000,309.000000,1.000000,18.000000,26.600000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,210.300000,9.300000,393.000000,1.000000,21.000000,28.600000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,275.800000,14.400000,710.000000,2.000000,36.000000,35.000000,5.700000,2.550000,3.400000,1.810000,1.580000,1.560000
max,6781.500000,83.500000,8556.000000,5.000000,62.000000,102.200000,52.130000,13.040000,13.960000,11.270000,8.100000,40.430000


In [15]:
finaltest_df.describe()

,Building0,Building1,Building3,Building4,Building5,Building6,Solar0,Solar1,Solar2,Solar3,Solar4,Solar5
count,107144.000000,66251.000000,154102.000000,29637.000000,12673.000000,45248.000000,21064.000000,67244.000000,52264.000000,52264.000000,52264.000000,65804.000000
mean,224.572170,11.219393,514.682736,1.323717,24.635209,30.667680,6.246485,2.021127,1.987600,1.497808,1.182110,4.323415
std,133.017208,7.789192,271.490558,0.528007,10.960251,5.707388,11.361842,3.495909,3.241568,2.431512,1.958194,9.195823
min,0.100000,1.500000,85.000000,1.000000,1.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,164.500000,5.600000,308.000000,1.000000,18.000000,26.400000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,207.000000,9.300000,392.000000,1.000000,21.000000,28.600000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,274.500000,14.300000,697.000000,2.000000,35.000000,35.000000,7.372500,2.760000,3.470000,1.980000,1.640000,2.210000
max,6781.500000,83.500000,8556.000000,5.000000,62.000000,102.200000,53.270000,13.040000,13.960000,11.320000,8.100000,40.430000


In [24]:
finaltest_df1 = finaltest_df.tail(2880)

In [26]:
finaltest_df1.describe()

,Building0,Building1,Building3,Building4,Building5,Building6,Solar0,Solar1,Solar2,Solar3,Solar4,Solar5
count,2824.000000,2880.000000,2875.000000,1850.000000,960.000000,2874.000000,2880.000000,2880.000000,2880.000000,2880.000000,2880.000000,2880.000000
mean,141.771105,9.912500,403.660870,1.327568,24.534375,30.775922,12.606865,3.277649,3.251729,2.958368,1.924375,9.812125
std,60.075046,6.883706,160.340711,0.534122,9.047947,6.868007,16.259182,4.269398,4.259863,3.814295,2.526760,12.864373
min,0.100000,1.600000,210.000000,1.000000,1.000000,12.200000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,112.300000,4.900000,268.000000,1.000000,17.000000,26.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,154.000000,8.100000,348.000000,1.000000,19.000000,27.800000,2.735000,0.685000,0.660000,0.600000,0.375000,1.970000
75%,175.600000,13.000000,527.000000,2.000000,34.000000,36.000000,23.872500,6.230000,6.085000,5.752500,3.622500,18.677500
max,324.600000,38.300000,854.000000,4.000000,46.000000,73.600000,53.270000,12.730000,13.700000,11.320000,7.950000,40.410000


In [27]:
finaltest_df1.to_csv('/content/drive/MyDrive/Solar Farm Data/CSV data/NOV_test_data.csv')